In [ ]:
# Standard library imports
import os
import glob
import logging
import shutil
from multiprocessing import Pool, cpu_count, Manager

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import wfdb
import neurokit2 as nk
from tqdm import tqdm
from scipy.signal import lfilter, iirnotch, detrend, butter, filtfilt, savgol_filter, freqz
from scipy import signal, stats
from statsmodels.robust import mad
import pywt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Dense, LSTM, Masking, Conv1D, MaxPooling1D, Flatten, Conv2D, MaxPooling2D, 
                                     Input, add, Activation, BatchNormalization, GlobalAveragePooling2D, Dropout)
from tensorflow.keras.utils import plot_model
from keras.optimizers import Adam


# Define directories
nsrdb_dir = "C:/Users/alexc/Desktop/MasterThesis/data/nsrdb"  # replace with your actual directory
afdb_dir = "C:/Users/alexc/Desktop/MasterThesis/data/afdb"  # replace with your actual directory
mitdb_dir = "C:/Users/alexc/Desktop/MasterThesis/data/mitdb"  # replace with your actual directory

results_dir = "C:/Users/alexc/Desktop/MasterThesis/results/"  # replace with your actual directory
preprocessing_dir = 'C:/Users/alexc/Desktop/MasterThesis/preprocessing/'  # replace with your actual directory

sec_segments_dir = preprocessing_dir  # replace with your actual directory
beats_segments_dir = preprocessing_dir # replace with your actual directory
RR_segments_dir = preprocessing_dir # replace with your actual directory
pwave_segments_dir = preprocessing_dir  # replace with your actual directory

sample_rate = 360 # Sampling rate
seconds = 10 # Number of seconds to be extracted from each segment
RRs = 301 # Number of RRs to be extracted from each segment
n_beats = 100 # Number of beats to be extracted from each segment
overlapping = 50 # Overlapping added to each segment
lowcut = 0.5  # Low cut-off frequency (Hz)
highcut = 50.0  # High cut-off frequency (Hz)
min_mV = 0 # Minimum amplitude (mV)
max_mV = 1 # Maximum amplitude (mV)
n_epochs = 10 # Number of epochs

#####################
### LOOP ELEMENTS ###
#####################  
analyse_list = ['seconds','beats','RR','pwave']
loop_list = ['manual','neurokit2','raw']
quality_list = ['excellent']
limited_time_list = [60, 300, 600] # minutes 

# Set up logging
# Try catch to ignore the flushing error when the file do not exists
try:
    logging.basicConfig(filename='C:/Users/alexc/Desktop/MasterThesis/logs.log', level=logging.INFO, flushing=True)
except:
    logging.basicConfig(filename='C:/Users/alexc/Desktop/MasterThesis/logs.log', level=logging.INFO, flushing=True)

## PLOTTING AND ELSE

In [ ]:
# Save and plot the dropped segments percentages per database
def plot_segment_percentages(results_subdir, ignored_counts, total_counts):
    databases = list(total_counts.keys())
    ignored_percentages = [ignored_counts[db] / total_counts[db] * 100 for db in databases]
    fig, ax = plt.subplots()
    ax.bar(databases, ignored_percentages)
    ax.set_xlabel('Database')
    ax.set_ylabel('Percentage of Ignored Segments (%)')
    ax.set_title('Percentage of Ignored Segments per Database')
    fig.savefig(os.path.join(results_subdir, 'ignored_segments_plot.png'))
    plt.close(fig)
    df = pd.DataFrame({
        'Database': databases,
        'Ignored Counts': [ignored_counts[db] for db in databases],
        'Total Counts': [total_counts[db] for db in databases],
        'Ignored Percentages': ignored_percentages
    })
    df.to_csv(os.path.join(results_subdir, 'ignored_segments.csv'), index=False)

# Plot the padding percentages per database, used by Beats methodology
def plot_padding_percentages(padding_percentages, results_subdir):
    fig, ax = plt.subplots()
    ax.set_title('Padding Percentages')
    ax.set_ylabel('Percentage')
    ax.set_xticks(range(len(padding_percentages)))
    ax.set_xticklabels(padding_percentages.keys())
    ax.bar(padding_percentages.keys(), padding_percentages.values())
    save_plot(fig, results_subdir, "padding_percentages")

# Plot the confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues, results_subdir=None, fileName=None):
    plt.figure(figsize = (5,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, cbar=False, xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(os.path.join(results_subdir, f'{fileName}_confusion_matrix.png'))
    plt.close()

# Save a plot for a given figure, path and filename
def save_plot(fig, path, filename):
    os.makedirs(path, exist_ok=True)
    fig.savefig(os.path.join(path, f"{filename}.png"), dpi=300)
    plt.close(fig)

# Save the metrics and confusion matrix to a CSV file
# If the file already exists, append to it
def save_metrics_to_csv(metrics, cm, filename, model_info):
    metrics_df = pd.DataFrame(metrics, index=[0])
    cm_df = pd.DataFrame(cm.flatten(), index=['TN', 'FP', 'FN', 'TP']).T
    df = pd.concat([metrics_df, cm_df], axis=1)
    df['Model'] = model_info
    if os.path.isfile(filename):
        df.to_csv(filename, mode='a', header=False, index=False)
    else:
        df.to_csv(filename, mode='w', header=True, index=False)

# Collect all results from the results directory and save them to a single CSV file
def collect_results(results_dir):
    df_list = []
    for csv_file in glob.glob(results_dir + '/*/*.csv'):
        folder_name = os.path.dirname(csv_file).replace("\\", "/")
        folder_name_parts = folder_name.split('/')[-1].split('_')
        analyse_type = folder_name_parts[0].replace("results", "")
        loop_type = folder_name_parts[1]
        quality_type = '_'.join(folder_name_parts[2:])
        df = pd.read_csv(csv_file)
        df['Analyse Type'] = analyse_type
        df['Loop Type'] = loop_type
        df['Quality Type'] = quality_type
        df_list.append(df)

    complete_results = pd.concat(df_list, ignore_index=True)
    column_order = ['Analyse Type', 'Loop Type', 'Quality Type', 'Model', 'accuracy', 'specificity', 'f1_score', 'TN', 'FP', 'FN', 'TP']
    complete_results = complete_results[column_order]
    complete_results.sort_values(by='accuracy', ascending=False, inplace=True)
    complete_results.to_csv(os.path.join(results_dir, 'complete_results.csv'), index=False)

# Plot the results from the complete results CSV file
# We plot the top 10 configurations by accuracy
def plot_results(df, results_dir):
    df["Configuration"] = df["Analyse Type"] + "_" + df["Loop Type"] + "_" + df["Quality Type"] + "_" + df["Model"]
    top_configurations = df.nlargest(10, "accuracy")
    plt.figure(figsize=(10, 6))
    sns.barplot(x="Configuration", y="accuracy", data=top_configurations)
    plt.xticks(rotation=90) 
    plt.title("Top 10 Configurations by Accuracy")
    plt.ylabel("Accuracy")
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, "results_plot.png"))
    plt.close()

# First analyse of the data made after preprocessing
def data_analysis(directory, output_directory):
    subfolders = [f.path for f in os.scandir(directory) if f.is_dir()]
    def count_segments_in_folder(folder):
        return len([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])
    segment_counts = {os.path.basename(folder): count_segments_in_folder(folder) for folder in subfolders}
    df = pd.DataFrame.from_dict(segment_counts, orient='index', columns=['segment_count'])
    df.index.name = 'database'
    df.to_csv(os.path.join(output_directory, 'segment_counts.csv'), index=True)



## PREPROCESSING FUNCTIONS

In [ ]:
# Remove subfolders based on the directory path
def remove_subfolders(directory):
    if os.path.exists(directory):
        for item in os.scandir(directory):
            if item.is_dir():
                shutil.rmtree(item.path)

# Load data and labels from the files in the directory
def load_data_and_labels(files, directory):
    data = [np.loadtxt(os.path.join(directory, file), delimiter=',') for file in files]
    labels = [int(file.split('_')[-1][0]) for file in files]  # Extract class label from filename
    return data, labels

# Rescale the data to the new min and max
def rescale_data(data, new_min, new_max):
    current_min = np.min(data)
    current_max = np.max(data)
    rescaled_data = (data - current_min) * (new_max - new_min) / (current_max - current_min) + new_min
    return rescaled_data

# Rescale the 2D data to the new min and max, used by Peaks
def rescale_data_2D(data, new_min, new_max):
    current_min = np.min(data, axis=0)
    current_max = np.max(data, axis=0)
    rescaled_data = (data - current_min) * (new_max - new_min) / (current_max - current_min) + new_min
    return rescaled_data

# Function to apply a bandpass filter to the data
# https://scipy-cookbook.readthedocs.io/items/ButterworthBandpass.html
def apply_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Function to apply a lowpass filter to the data
# https://notebook.community/CSchoel/learn-wavelets/wavelet-denoising
# Sigma = constant scale factor of normal distribution * MAD
def wavelet_denoising(data, wavelet='db5', level=1):
    coeff = pywt.wavedec(data, wavelet, mode="per")
    sigma = (1/0.6745) * mad(coeff[-level])
    uthresh = sigma * np.sqrt(2*np.log(len(data)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='soft') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')

# Calculate the padding percentage added to the data, used by Beats
def calculate_padding_percentage(data, pad_value=-1):
    padding_percentages = []
    for array in data:
        num_total_values = np.prod(array.shape)
        num_padding_values = np.count_nonzero(array == pad_value)
        padding_percentage = (num_padding_values / num_total_values) * 100
        padding_percentages.append(padding_percentage)
    return np.mean(padding_percentages)

# Evaluate the model, compute the metrics and save the results to a CSV file
def evaluate_model(model, model_name, train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir):
    model.fit(train_data, train_labels, epochs=n_epochs, validation_data=(val_data, val_labels))
    # Evaluate the model on the test data and retrieve the metrics
    test_predictions = (model.predict(test_data) > 0.5).astype("int32")
    accuracy = accuracy_score(test_labels, test_predictions)
    cm = confusion_matrix(test_labels, test_predictions)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(test_labels, test_predictions)
    metrics = {
        "accuracy": accuracy,
        "specificity": specificity,
        "f1_score": f1,
    }
    save_metrics_to_csv(metrics, cm, os.path.join(results_subdir, 'performance_metrics.csv'), model_name)
    plot_confusion_matrix(cm, ['NS', 'AF'], results_subdir=results_subdir, fileName=model_name)
    # Plot the ROC and AUC
    fpr, tpr, _ = roc_curve(val_labels, model.predict(val_data))
    roc_auc = auc(fpr, tpr)
    roc_df = pd.DataFrame({
        'FPR': fpr,
        'TPR': tpr,
        'AUC': [roc_auc]*len(fpr) 
    })
    file_name = os.path.join(results_subdir, f'roc_auc_{model_name}.csv')
    roc_df.to_csv(file_name, index=False)

    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.savefig(os.path.join(results_subdir, f'{model_name}_roc_curve.png')) 
    plt.close()

# Evaluate the model for RandomForest, compute the metrics and save the results to a CSV file
def evaluate_model_RF(model, model_name, train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir):
    # Flatten the data because RandomForestClassifier cannot handle 3D data
    train_data_flattened = train_data.reshape(train_data.shape[0], -1)
    val_data_flattened = val_data.reshape(val_data.shape[0], -1)
    test_data_flattened = test_data.reshape(test_data.shape[0], -1)
    
    model.fit(train_data_flattened, train_labels)

    # Evaluate the model on the test data and retrieve the metrics
    test_predictions = model.predict(test_data_flattened)
    accuracy = accuracy_score(test_labels, test_predictions)
    cm = confusion_matrix(test_labels, test_predictions)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(test_labels, test_predictions)
    metrics = {
        "accuracy": accuracy,
        "specificity": specificity,
        "f1_score": f1,
    }
    save_metrics_to_csv(metrics, cm, os.path.join(results_subdir, 'performance_metrics.csv'), model_name)
    plot_confusion_matrix(cm, ['NS', 'AF'], results_subdir=results_subdir, fileName=model_name)

    # Plot ROC and AUC
    fpr, tpr, _ = roc_curve(val_labels, model.predict_proba(val_data_flattened)[:, 1]) # Probabilities are needed for ROC curve
    roc_auc = auc(fpr, tpr)
    roc_df = pd.DataFrame({
        'FPR': fpr,
        'TPR': tpr,
        'AUC': [roc_auc]*len(fpr)
    })
    file_name = os.path.join(results_subdir, f'roc_auc_{model_name}.csv')
    roc_df.to_csv(file_name, index=False)

    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.savefig(os.path.join(results_subdir, f'{model_name}_roc_curve.png'))  
    plt.close()  

# Define the ResNet block for 1D data as described in the paper
# https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(19)31721-0/fulltext
def res_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True, pool_size=2):
    shortcut = x
    if conv_shortcut:
        shortcut = layers.Conv1D(filters, 1, strides=stride)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(pool_size)(x)
    return x

# Create and return the ResNet1D model as described in the paper
# https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(19)31721-0/fulltext
def ResNet1D(length, features):
    inputs = Input(shape=(length,features))
    x = layers.Masking(mask_value=-1.0)(inputs)
    for _ in range(10):  # 10 residual blocks
        x = res_block(x, 64, pool_size=1) 
    x = layers.Dropout(0.5)(x)
    x = layers.MaxPooling1D(2)(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs, outputs)

In [ ]:
# Split the ECG files into train, val, and test sets with a 70/10/20 split.
def split_files(files, source):
    num_total_files = len(files)
    train_end_idx = int(0.7 * num_total_files)
    val_end_idx = int(0.8 * num_total_files)
    
    train_files = files[:train_end_idx]
    val_files = files[train_end_idx:val_end_idx]
    test_files = files[val_end_idx:]
    
    train_df = pd.DataFrame({'fileName': train_files, 'source': [source]*len(train_files), 'set': 'train'})
    val_df = pd.DataFrame({'fileName': val_files, 'source': [source]*len(val_files), 'set': 'val'})
    test_df = pd.DataFrame({'fileName': test_files, 'source': [source]*len(test_files), 'set': 'test'})
        
    return pd.concat([train_df, val_df, test_df], axis=0).reset_index(drop=True)

def allocate_ecg_to_datasets(nsrdb_dir, afdb_dir, mitdb_dir):
    datasets = {
        "nsrdb": nsrdb_dir,
        "afdb": afdb_dir,
        "mitdb": mitdb_dir
    }
        
    all_dfs = []
    for source, directory in datasets.items():
        files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".dat")]
        np.random.shuffle(files)
        df = split_files(files, source)
        all_dfs.append(df)
    
    result_df = pd.concat(all_dfs, axis=0).reset_index(drop=True)
    
    return result_df

ecg_df = allocate_ecg_to_datasets(nsrdb_dir, afdb_dir, mitdb_dir)
ecg_df.to_csv("C:/Users/alexc/Desktop/MasterThesis/splits.csv", index=False)
print(ecg_df)

# SECONDS

SECONDS PREPROCESSING

Based on the article by Attia et al.

https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(19)31721-0/fulltext

In [ ]:
def process_ecg_data_seconds(directory, segments_directory, subfolder, is_nsrdb=False, is_afdb=False, is_mitdb=False, patient_id=None, seconds=10, limited_time=None, loop_type='manual', quality_type='excellent'):
    total_segments = 0
    ignored_segments = 0

    if patient_id is not None:
        files = [file for file in os.listdir(directory) if file.startswith(patient_id)]
    else:
        files = os.listdir(directory)

    for file in tqdm(files):
        if file.endswith(".dat"):
            logging.info(f"Processing file: {file}")

            # Determine where to save the file based on the dataframe
            current_set = ecg_df.loc[ecg_df['fileName'] == os.path.join(directory, file), 'set'].values[0]
            save_dir = os.path.join(segments_directory, current_set)

            record = wfdb.rdrecord(os.path.join(directory, file[:-4]))

            # Identify the index of the desired lead
            if is_afdb :
                try:
                    lead_index = record.sig_name.index('ECG1')
                except ValueError:
                    print(f"No ECG2 lead in {file}. Skipping this file.")
                    continue
            elif is_nsrdb:
                try:
                    lead_index = record.sig_name.index('ECG1')
                except ValueError:
                    print(f"No ECG1 lead in {file}. Skipping this file.")
                    continue
            elif is_mitdb:
                try:
                    lead_index = record.sig_name.index('MLII')
                except ValueError:
                    print(f"No MLII lead in {file}. Skipping this file.")
                    continue

            ecg_signals = record.p_signal[:, lead_index]
            window_length = record.fs * seconds  # seconds parameter window

            cleaned_ecg = ecg_signals
            # If we work with limited time
            if limited_time != None:
                cleaned_ecg = ecg_signals[:record.fs * 60 * limited_time]
                
            try:
                if is_afdb or is_mitdb:
                    logging.info(f"Processing AFDB or MITDB data for file: {file}")
                    annotation_extension = 'qrs' if is_afdb else 'atr'
                    qrs_annotations = wfdb.rdann(os.path.join(directory, file[:-4]), annotation_extension)
                    i = 0
                    while i < len(qrs_annotations.sample) - 1 and qrs_annotations.sample[i] < len(cleaned_ecg):
                        if qrs_annotations.symbol[i] == 'N':
                            start = qrs_annotations.sample[i]
                            end = min(len(cleaned_ecg), start + window_length)  # 10 seconds after the QRS complex
                            window_annotation_indices = [i for i in range(len(qrs_annotations.sample)) if start <= qrs_annotations.sample[i] < end]
                            next_symbols = [qrs_annotations.symbol[i] for i in window_annotation_indices]
                            if all(symbol == 'N' for symbol in next_symbols):
                                # Extract the window
                                window = cleaned_ecg[start:end]
                                if True: #not np.any(window == 0): not used because always true
                                    # Check if the window is exactly the window_length
                                    if len(window) == window_length:
                                        #################
                                        ### Loop type ###
                                        #################
                                        window = nk.ecg_invert(window, sampling_rate=record.fs)[0]
                                        window = nk.signal_resample(window, method="FFT", sampling_rate=record.fs, desired_sampling_rate=sample_rate)
                                        if loop_type == 'manual':
                                            window = apply_bandpass_filter(window, lowcut, highcut, sample_rate, order=5)
                                            window = wavelet_denoising(window)
                                        elif loop_type == 'neurokit2':
                                            window = nk.ecg_clean(window, sampling_rate=sample_rate, method='neurokit2')

                                        window = rescale_data(window, min_mV, max_mV)
                                        ##################
                                        ## Quality type ##
                                        ##################
                                        # If we only retrieve the Excellent quality segments
                                        if quality_type == 'excellent':
                                            quality = nk.ecg_quality(window, sampling_rate=sample_rate, method="zhao2018", approach="fuzzy")
                                            if quality == 'Excellent':
                                                # Save the window to a file
                                                np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_1.csv"), window, delimiter=",")
                                            else:
                                                ignored_segments += 1
                                        # If we retrieve all the segments
                                        else:
                                            np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_1.csv"), window, delimiter=",")

                                i += overlapping
                                total_segments += 1
                            else:
                                i += 1
                        else:
                            i += 1
                else:
                    logging.info(f"Processing NSRDB data for file: {file}")
                    # Split the cleaned signal into 10-second segments (window_length points at sample_rate Hz)
                    segments = np.array_split(cleaned_ecg, len(cleaned_ecg) // window_length, axis=0)
                    for i, segment in enumerate(segments):
                        if not np.any(segment == 0):
                            if len(segment) == window_length:
                                #################
                                ### Loop type ###
                                #################
                                total_segments += 1
                                segment = nk.ecg_invert(segment, sampling_rate=record.fs)[0]
                                segment = nk.signal_resample(segment, method="FFT", sampling_rate=record.fs, desired_sampling_rate=sample_rate)
                                if loop_type == 'manual':
                                    segment = apply_bandpass_filter(segment, lowcut, highcut, sample_rate, order=5)
                                    segment = wavelet_denoising(segment)
                                elif loop_type == 'neurokit2':
                                    segment = nk.ecg_clean(segment, sampling_rate=sample_rate, method='neurokit2')

                                segment = rescale_data(segment, min_mV, max_mV)
                                    
                                ##################
                                ## Quality type ##
                                ##################
                                if quality_type == 'excellent':
                                    quality = nk.ecg_quality(segment, sampling_rate=sample_rate, method="zhao2018", approach="fuzzy")
                                    if quality == 'Excellent':
                                        np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_0.csv"), segment, delimiter=",")
                                    else:
                                        ignored_segments += 1
                                else:
                                    np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_0.csv"), segment, delimiter=",")

            except Exception as e:
                print(e)
                logging.error(f"Exception occurred: {e}")
    
    return total_segments, ignored_segments

SECONDS MODELS

First load the data and labels from the CSV files, then prepare the data for the models and finally run the models

In [ ]:
def sec_run_models(seconds=10, results_subdir=None, actual_folder=None):
    train_dir = os.path.join(preprocessing_dir, actual_folder, 'train')
    val_dir = os.path.join(preprocessing_dir, actual_folder, 'val')
    test_dir = os.path.join(preprocessing_dir, actual_folder, 'test')

    train_files = os.listdir(train_dir)
    val_files = os.listdir(val_dir)
    test_files = os.listdir(test_dir)

    train_data, train_labels = load_data_and_labels(train_files, train_dir)
    val_data, val_labels = load_data_and_labels(val_files, val_dir)
    test_data, test_labels = load_data_and_labels(test_files, test_dir)

    frequency_used = sample_rate * seconds
    train_data = np.array(train_data).reshape(-1, frequency_used, 1)
    val_data = np.array(val_data).reshape(-1, frequency_used, 1)
    test_data = np.array(test_data).reshape(-1, frequency_used, 1)
    
    train_labels = np.array(train_labels).astype('float32')
    val_labels = np.array(val_labels).astype('float32')
    test_labels = np.array(test_labels).astype('float32')

    # Shuffle data
    train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
    val_data, val_labels = shuffle(val_data, val_labels, random_state=42)
    test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

    # Apply undersampling on train data
    rus = RandomUnderSampler(sampling_strategy='majority', random_state=42)
    train_data, train_labels = rus.fit_resample(train_data.reshape(train_data.shape[0], -1), train_labels)
    train_data = train_data.reshape(-1, frequency_used, 1)

    ###########
    ### CNN ###
    ###########
    model = Sequential()
    model.add(Masking(mask_value=-1., input_shape=(train_data.shape[1], train_data.shape[2])))
    model.add(Conv1D(64, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(128, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(256, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    evaluate_model(model, 'CNN', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)

    ############
    ### LSTM ###
    ############
    model = Sequential()
    model.add(Masking(mask_value=-1., input_shape=(train_data.shape[1], train_data.shape[2])))
    model.add(LSTM(64)) 
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(Dense(1, activation='sigmoid')) 
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    evaluate_model(model, 'LSTM', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)

    # #####################
    # ### RANDOM FOREST ###
    # #####################
    model = RandomForestClassifier(n_estimators=200, random_state=42)
    evaluate_model_RF(model, "RandomForest", train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)
    
    ##############
    ### ResNet ###
    ##############
    model = ResNet1D(train_data.shape[1], train_data.shape[2])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    evaluate_model(model, 'ResNet', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)


## BEATS

BEATS PREPROCESSING

In [ ]:
def process_ecg_data_beats(directory, segments_directory, subfolder, is_nsrdb=False, is_afdb=False, is_mitdb=False, patient_id=None, n_beats=10, limited_time=None, loop_type='manual', quality_type='excellent'):
    total_segments = 0
    ignored_segments = 0
    if patient_id is not None:
        files = [file for file in os.listdir(directory) if file.startswith(patient_id)]
    else:
        files = os.listdir(directory)

    for file in tqdm(files):
        if file.endswith(".dat"):
            logging.info(f"Processing file: {file}")

            # Determine where to save the file based on the dataframe
            current_set = ecg_df.loc[ecg_df['fileName'] == os.path.join(directory, file), 'set'].values[0]
            save_dir = os.path.join(segments_directory, current_set)

            record = wfdb.rdrecord(os.path.join(directory, file[:-4]))

            if is_afdb :
                try:
                    lead_index = record.sig_name.index('ECG1')
                except ValueError:
                    print(f"No ECG2 lead in {file}. Skipping this file.")
                    continue
            elif is_nsrdb:
                try:
                    lead_index = record.sig_name.index('ECG1')
                except ValueError:
                    print(f"No ECG1 lead in {file}. Skipping this file.")
                    continue
            elif is_mitdb:
                try:
                    lead_index = record.sig_name.index('MLII')
                except ValueError:
                    print(f"No MLII lead in {file}. Skipping this file.")
                    continue

            ecg_signals = record.p_signal[:, lead_index]
            cleaned_ecg = ecg_signals
            # If we work with limited time
            if limited_time != None:
                cleaned_ecg = ecg_signals[:record.fs * 60 * limited_time] # for now only work with limited time

            try:
              if is_afdb or is_mitdb:
                annotation_extension = 'qrs' if is_afdb else 'atr'
                qrs_annotations = wfdb.rdann(os.path.join(directory, file[:-4]), annotation_extension)
                i = 0
                while i < len(qrs_annotations.sample) - n_beats: # Ensure there are at least n_beats left
                    if qrs_annotations.symbol[i] == 'N':
                        next_symbols = qrs_annotations.symbol[i+1: i+n_beats]  
                        if all(symbol == 'N' for symbol in next_symbols):
                            start = qrs_annotations.sample[i]
                            end = qrs_annotations.sample[i+n_beats]
                            # Extract the window
                            window = cleaned_ecg[start:end]
                            if len(window) > 0:
                                #################
                                ### Loop type ###
                                #################
                                window = nk.ecg_invert(window, sampling_rate=record.fs)[0]
                                window = nk.signal_resample(window, method="FFT", sampling_rate=record.fs, desired_sampling_rate=sample_rate)
                                if loop_type == 'manual':
                                    window = apply_bandpass_filter(window, lowcut, highcut, sample_rate, order=5)
                                    window = wavelet_denoising(window)
                                elif loop_type == 'neurokit2':
                                    window = nk.ecg_clean(window, sampling_rate=sample_rate, method='neurokit2')

                                window = rescale_data(window, min_mV, max_mV)
                                ##################
                                ## Quality type ##
                                ##################
                                # If we only retrieve the Excellent quality segments
                                if quality_type == 'excellent':
                                    quality = nk.ecg_quality(window, sampling_rate=sample_rate, method="zhao2018", approach="fuzzy")
                                    if quality == 'Excellent':
                                        # Save the window to a file
                                        np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_1.csv"), window, delimiter=",")
                                    else:
                                        ignored_segments += 1
                                # If we retrieve all the segments
                                else:
                                    np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_1.csv"), window, delimiter=",")
                            
                            i += overlapping
                            total_segments += 1
                            continue
                    i += 1
              else:
                qrs_annotations = wfdb.rdann(os.path.join(directory, file[:-4]), 'atr')
                i = 0
                while i < len(qrs_annotations.sample) - n_beats: # Ensure there are at least n_beats left
                    next_symbols = qrs_annotations.symbol[i+1: i+n_beats] 
                    if all(symbol == 'N' for symbol in next_symbols):
                        start = qrs_annotations.sample[i]
                        end = qrs_annotations.sample[i+n_beats] 
                        # Extract the window
                        window = cleaned_ecg[start:end]
                        # Check if the window is empty
                        if len(window) > 0:
                            #################
                            ### Loop type ###
                            #################
                            segment = nk.ecg_invert(window, sampling_rate=record.fs)[0]
                            segment = nk.signal_resample(segment, method="FFT", sampling_rate=record.fs, desired_sampling_rate=sample_rate)
                            if loop_type == 'manual':
                                segment = apply_bandpass_filter(segment, lowcut, highcut, sample_rate, order=5)
                                segment = wavelet_denoising(segment)
                            elif loop_type == 'neurokit2':
                                segment = nk.ecg_clean(segment, sampling_rate=sample_rate, method='neurokit2')

                            segment = rescale_data(segment, min_mV, max_mV)
                                
                            ##################
                            ## Quality type ##
                            ##################
                            if quality_type == 'excellent':
                                quality = nk.ecg_quality(segment, sampling_rate=sample_rate, method="zhao2018", approach="fuzzy")
                                if quality == 'Excellent':
                                    np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_0.csv"), segment, delimiter=",")
                                else:
                                    ignored_segments += 1
                            else:
                                np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_0.csv"), segment, delimiter=",")
                        
                        i += overlapping
                        total_segments += 1
                        continue
                    i += 1
            except Exception as e:
              logging.error(f"Exception occurred: {e}")
    return total_segments, ignored_segments

BEATS MODELS

In [ ]:
def beats_run_models(results_subdir=None, actual_folder=None):
    train_dir = os.path.join(preprocessing_dir, actual_folder, 'train')
    val_dir = os.path.join(preprocessing_dir, actual_folder, 'val')
    test_dir = os.path.join(preprocessing_dir, actual_folder, 'test')

    train_files = os.listdir(train_dir)
    val_files = os.listdir(val_dir)
    test_files = os.listdir(test_dir)

    train_data, train_labels = load_data_and_labels(train_files, train_dir)
    val_data, val_labels = load_data_and_labels(val_files, val_dir)
    test_data, test_labels = load_data_and_labels(test_files, test_dir)

    # Determine the maximum length of the sequences
    max_len = max(max(len(x) for x in train_data), max(len(x) for x in val_data), max(len(x) for x in test_data))
    # Pad the sequences in each data array with -1
    train_data = [np.pad(x, (0, max_len - len(x)), constant_values=-1) for x in train_data]
    val_data = [np.pad(x, (0, max_len - len(x)), constant_values=-1) for x in val_data]
    test_data = [np.pad(x, (0, max_len - len(x)), constant_values=-1) for x in test_data]
    afdb_padding_percentage = calculate_padding_percentage(train_data)
    mitdb_padding_percentage = calculate_padding_percentage(val_data)
    nsrdb_padding_percentage = calculate_padding_percentage(test_data)
    padding_percentages = {
        "TRAIN": afdb_padding_percentage,
        "VAL": mitdb_padding_percentage,
        "TEST": nsrdb_padding_percentage
    }
    plot_padding_percentages(padding_percentages, results_subdir)

    train_data = np.array(train_data).reshape(-1, max_len, 1)
    val_data = np.array(val_data).reshape(-1, max_len, 1)
    test_data = np.array(test_data).reshape(-1, max_len, 1)
    
    train_labels = np.array(train_labels).astype('float32')
    val_labels = np.array(val_labels).astype('float32')
    test_labels = np.array(test_labels).astype('float32')

    # Shuffle data
    train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
    val_data, val_labels = shuffle(val_data, val_labels, random_state=42)
    test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

    # Apply undersampling on train data
    rus = RandomUnderSampler(sampling_strategy='majority', random_state=42)
    train_data, train_labels = rus.fit_resample(train_data.reshape(train_data.shape[0], -1), train_labels)
    train_data = train_data.reshape(-1, max_len, 1)

    ###########
    ### CNN ###
    ###########
    model = Sequential()
    model.add(Masking(mask_value=-1., input_shape=(max_len, 1)))
    model.add(Conv1D(64, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(128, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(256, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    evaluate_model(model, 'CNN', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)

    ############
    ### LSTM ###
    ############
    model = Sequential()
    model.add(Masking(mask_value=-1., input_shape=(max_len, 1)))
    model.add(LSTM(64)) 
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(Dense(1, activation='sigmoid')) 
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    evaluate_model(model, 'LSTM', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)

    #####################
    ### RANDOM FOREST ###
    #####################
    model = RandomForestClassifier(n_estimators=200, random_state=42)
    evaluate_model_RF(model, "RandomForest", train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)
    
    ##############
    ### ResNet ###
    ##############
    model = ResNet1D(max_len,1)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    evaluate_model(model, 'ResNet', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)

## RR 

RR PREPROCESSING

Based on the article by Duan et al. 

https://pubmed.ncbi.nlm.nih.gov/35925979/

In [ ]:
def process_ecg_data_RR(directory, segments_directory, subfolder, is_nsrdb=False, is_afdb=False, is_mitdb=False, patient_id=None, RRs=10, limited_time=None):
    if patient_id is not None:
        files = [file for file in os.listdir(directory) if file.startswith(patient_id)]
    else:
        files = os.listdir(directory)

    for file in tqdm(files):
        if file.endswith(".dat"):
            logging.info(f"Processing file: {file}")

            # Determine where to save the file based on the dataframe
            current_set = ecg_df.loc[ecg_df['fileName'] == os.path.join(directory, file), 'set'].values[0]
            save_dir = os.path.join(segments_directory, current_set)

            record = wfdb.rdrecord(os.path.join(directory, file[:-4]))

            if is_afdb :
                try:
                    lead_index = record.sig_name.index('ECG1')
                except ValueError:
                    print(f"No ECG2 lead in {file}. Skipping this file.")
                    continue
            elif is_nsrdb:
                try:
                    lead_index = record.sig_name.index('ECG1')
                except ValueError:
                    print(f"No ECG1 lead in {file}. Skipping this file.")
                    continue
            elif is_mitdb:
                try:
                    lead_index = record.sig_name.index('MLII')
                except ValueError:
                    print(f"No MLII lead in {file}. Skipping this file.")
                    continue

            ecg_signals = record.p_signal[:, lead_index]
            cleaned_ecg = ecg_signals
            # If we work with limited time
            if limited_time != None:
                cleaned_ecg = ecg_signals[:record.fs * 60 * limited_time] # for now only work with limited time

            try:
              if is_afdb or is_mitdb:
                logging.info(f"Processing AFDB or MITDB data for file: {file}")
                annotation_extension = 'qrs' if is_afdb else 'atr'
                qrs_annotations = wfdb.rdann(os.path.join(directory, file[:-4]), annotation_extension)
                i = 0
                while i < len(qrs_annotations.sample) - RRs and qrs_annotations.sample[i] < len(cleaned_ecg):
                    if qrs_annotations.symbol[i] == 'N':
                        next_symbols = qrs_annotations.symbol[i+1: i+RRs]
                        if all(symbol == 'N' for symbol in next_symbols):
                            start = qrs_annotations.sample[i]
                            end = qrs_annotations.sample[i+RRs]
                            # Locate the indices of R-peaks that fall within the window
                            rpeaks = [i for i in qrs_annotations.sample if start <= i < end]
                            window = np.diff(rpeaks)
                            # Resample to the frequency to get ms
                            window = window / record.fs
                            if len(window) == RRs-1:
                                np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_1.csv"), window, delimiter=",")
                    
                            i+=overlapping
                        else:
                            i += 1
                    i += 1
              else:
                  logging.info(f"Processing NSRDB data for file: {file}")
                  qrs_annotations = wfdb.rdann(os.path.join(directory, file[:-4]), 'atr')
                  i = 0
                  while i < len(qrs_annotations.sample) - RRs and qrs_annotations.sample[i] < len(cleaned_ecg):
                    if qrs_annotations.symbol[i] == 'N':
                        next_symbols = qrs_annotations.symbol[i+1: i+RRs]
                        if all(symbol == 'N' for symbol in next_symbols):
                          start = qrs_annotations.sample[i]
                          end = qrs_annotations.sample[i+RRs]
                          # Locate the indices of R-peaks that fall within the window
                          rpeaks = [i for i in qrs_annotations.sample if start <= i < end]
                          segment = np.diff(rpeaks)
                          # Resample to the frequency to get ms
                          segment = segment / record.fs
                          if len(segment) == RRs-1:
                              np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_0.csv"), segment, delimiter=",")
                          
                          i += overlapping
                        else:
                            i += 1
                    i += 1
            except Exception as e:
              print(e)
              logging.error(f"Exception occurred: {e}")

RR MODELS

In [ ]:
def RR_run_models(results_subdir=None, actual_folder=None):
    train_dir = os.path.join(preprocessing_dir, actual_folder, 'train')
    val_dir = os.path.join(preprocessing_dir, actual_folder, 'val')
    test_dir = os.path.join(preprocessing_dir, actual_folder, 'test')

    train_files = os.listdir(train_dir)
    val_files = os.listdir(val_dir)
    test_files = os.listdir(test_dir)

    train_data, train_labels = load_data_and_labels(train_files, train_dir)
    val_data, val_labels = load_data_and_labels(val_files, val_dir)
    test_data, test_labels = load_data_and_labels(test_files, test_dir)

    train_data = np.array(train_data).reshape(-1, RRs-1, 1)
    val_data = np.array(val_data).reshape(-1, RRs-1, 1)
    test_data = np.array(test_data).reshape(-1, RRs-1, 1)
    
    train_labels = np.array(train_labels).astype('float32')
    val_labels = np.array(val_labels).astype('float32')
    test_labels = np.array(test_labels).astype('float32')

    # Shuffle data
    train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
    val_data, val_labels = shuffle(val_data, val_labels, random_state=42)
    test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

    # Apply undersampling on train data
    rus = RandomUnderSampler(sampling_strategy='majority', random_state=42)
    train_data, train_labels = rus.fit_resample(train_data.reshape(train_data.shape[0], -1), train_labels)
    train_data = train_data.reshape(-1, RRs-1, 1)

    ###########
    ### CNN ###
    ###########
    model = Sequential()
    model.add(Masking(mask_value=-1., input_shape=(RRs-1,1)))
    model.add(Conv1D(64, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(128, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(256, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    evaluate_model(model, 'CNN', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)

    ############
    ### LSTM ###
    ############
    model = Sequential()
    model.add(Masking(mask_value=-1., input_shape=(RRs-1,1)))
    model.add(LSTM(64)) 
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(Dense(1, activation='sigmoid')) 
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    evaluate_model(model, 'LSTM', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)


    #####################
    ### RANDOM FOREST ###
    #####################
    model = RandomForestClassifier(n_estimators=200, random_state=42)
    evaluate_model_RF(model, "RandomForest", train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)
    
    ##############
    ### ResNet ###
    ##############
    model = ResNet1D(RRs-1,1) 
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    evaluate_model(model, 'ResNet', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)


# PEAKS

PEAKS PREPROCESSING

Based on the article of Castro et al. 

https://ijece.iaescore.com/index.php/IJECE/article/view/21303

In [ ]:
def process_ecg_data_pwave(directory, segments_directory, subfolder, is_nsrdb=False, is_afdb=False, is_mitdb=False, patient_id=None, RRs=10, limited_time=None):
    if patient_id is not None:
        files = [file for file in os.listdir(directory) if file.startswith(patient_id)]
    else:
        files = os.listdir(directory)

    for file in tqdm(files):
        if file.endswith(".dat"):
            logging.info(f"Processing file: {file}")

            # Determine where to save the file based on the dataframe
            current_set = ecg_df.loc[ecg_df['fileName'] == os.path.join(directory, file), 'set'].values[0]
            save_dir = os.path.join(segments_directory, current_set)

            record = wfdb.rdrecord(os.path.join(directory, file[:-4]))

            if is_afdb :
                try:
                    lead_index = record.sig_name.index('ECG1')
                except ValueError:
                    print(f"No ECG2 lead in {file}. Skipping this file.")
                    continue
            elif is_nsrdb:
                try:
                    lead_index = record.sig_name.index('ECG1')
                except ValueError:
                    print(f"No ECG1 lead in {file}. Skipping this file.")
                    continue
            elif is_mitdb:
                try:
                    lead_index = record.sig_name.index('MLII')
                except ValueError:
                    print(f"No MLII lead in {file}. Skipping this file.")
                    continue

            ecg_signals = record.p_signal[:, lead_index]
            cleaned_ecg = ecg_signals
            # If we work with limited time
            if limited_time != None:
                cleaned_ecg = ecg_signals[:record.fs * 60 * limited_time]

            try:
              if is_afdb or is_mitdb:
                  logging.info(f"Processing AFDB or MITDB data for file: {file}")
                  annotation_extension = 'qrs' if is_afdb else 'atr'
                  qrs_annotations = wfdb.rdann(os.path.join(directory, file[:-4]), annotation_extension)
                  i = 0
                  while i < len(qrs_annotations.sample) - RRs and qrs_annotations.sample[i] < len(cleaned_ecg):
                      if qrs_annotations.symbol[i] == 'N':
                          next_symbols = qrs_annotations.symbol[i+1: i+RRs]
                          if all(symbol == 'N' for symbol in next_symbols):
                              start = qrs_annotations.sample[i]
                              end = qrs_annotations.sample[i+RRs]
                              window = cleaned_ecg[start:end]
                              processed_ecg, info = nk.ecg_process(window, sampling_rate=record.fs)

                              all_peak_values = np.empty((RRs, 5))
                              all_peak_values.fill(0)
                              for peak_idx, peak_type in enumerate(['ECG_P_Peaks', 'ECG_Q_Peaks', 'ECG_R_Peaks', 'ECG_S_Peaks', 'ECG_T_Peaks']):
                                  peak_indices = np.where(processed_ecg[peak_type] == 1)[0]
                                  peak_values = processed_ecg['ECG_Clean'].iloc[peak_indices]
                                  if len(peak_values) == 0:
                                      all_peak_values[:, peak_idx] = 0
                                  else:
                                      num_peaks = min(len(peak_values), RRs)
                                      all_peak_values[:num_peaks, peak_idx] = peak_values[:num_peaks].values
                              all_peak_values = rescale_data_2D(all_peak_values, min_mV, max_mV)
                              np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_1.csv"), all_peak_values, delimiter=",")
                              i+=overlapping
                          else:
                              i += 1
                      else:
                          i += 1
              else:
                  logging.info(f"Processing NSRDB data for file: {file}")
                  qrs_annotations = wfdb.rdann(os.path.join(directory, file[:-4]), 'atr')
                  i = 0
                  while i < len(qrs_annotations.sample) - RRs and qrs_annotations.sample[i] < len(cleaned_ecg):  
                      if qrs_annotations.symbol[i] == 'N':
                          next_symbols = qrs_annotations.symbol[i+1: i+RRs]  
                          if all(symbol == 'N' for symbol in next_symbols):
                              start = qrs_annotations.sample[i]
                              end = qrs_annotations.sample[i+RRs]
                              window = cleaned_ecg[start:end]
                              processed_ecg, info = nk.ecg_process(window, sampling_rate=record.fs)

                              all_peak_values = np.empty((RRs, 5))
                              all_peak_values.fill(0) 
                              for peak_idx, peak_type in enumerate(['ECG_P_Peaks', 'ECG_Q_Peaks', 'ECG_R_Peaks', 'ECG_S_Peaks', 'ECG_T_Peaks']):
                                  peak_indices = np.where(processed_ecg[peak_type] == 1)[0]
                                  peak_values = processed_ecg['ECG_Clean'].iloc[peak_indices]
                                  if len(peak_values) == 0:
                                      all_peak_values[:, peak_idx] = 0
                                  else:
                                      num_peaks = min(len(peak_values), RRs)
                                      all_peak_values[:num_peaks, peak_idx] = peak_values[:num_peaks].values
                              all_peak_values = rescale_data_2D(all_peak_values, min_mV, max_mV)
                              np.savetxt(os.path.join(save_dir, f"{file[:-4]}_segment_{i}_0.csv"), all_peak_values, delimiter=",")
                              i += overlapping
                          else:
                              i += 1
                      else:
                          i += 1
 
            except Exception as e:
              print(e)
              logging.error(f"Exception occurred: {e}")

PEAKS MODELS

In [ ]:
def pwave_run_models(results_subdir=None, actual_folder=None):
    train_dir = os.path.join(preprocessing_dir, actual_folder, 'train')
    val_dir = os.path.join(preprocessing_dir, actual_folder, 'val')
    test_dir = os.path.join(preprocessing_dir, actual_folder, 'test')

    train_files = os.listdir(train_dir)
    val_files = os.listdir(val_dir)
    test_files = os.listdir(test_dir)

    train_data, train_labels = load_data_and_labels(train_files, train_dir)
    val_data, val_labels = load_data_and_labels(val_files, val_dir)
    test_data, test_labels = load_data_and_labels(test_files, test_dir)

    train_data = np.array(train_data).reshape(len(train_files), RRs, 5)
    val_data = np.array(val_data).reshape(len(val_files), RRs, 5)
    test_data = np.array(test_data).reshape(len(test_files), RRs, 5)
    
    train_labels = np.array(train_labels).astype('float32')
    val_labels = np.array(val_labels).astype('float32')
    test_labels = np.array(test_labels).astype('float32')

    # Shuffle data
    train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
    val_data, val_labels = shuffle(val_data, val_labels, random_state=42)
    test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

    # Apply undersampling on train data
    rus = RandomUnderSampler(sampling_strategy='majority', random_state=42)
    train_data, train_labels = rus.fit_resample(train_data.reshape(train_data.shape[0], -1), train_labels)
    train_data = train_data.reshape(-1, RRs, 5)

    # Replace nan values with zero in the data
    train_data = np.nan_to_num(train_data)
    val_data = np.nan_to_num(val_data)
    test_data = np.nan_to_num(test_data)

    ###########
    ### CNN ###
    ###########
    model = Sequential()
    model.add(Masking(mask_value=-1., input_shape=(train_data.shape[1], train_data.shape[2])))
    model.add(Conv1D(64, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(128, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(256, kernel_size=5, strides=1, activation='relu', padding='same'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    evaluate_model(model, 'CNN', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)

    ############
    ### LSTM ###
    ############
    model = Sequential()
    model.add(Masking(mask_value=-1., input_shape=(train_data.shape[1], train_data.shape[2])))
    model.add(LSTM(64)) 
    model.add(BatchNormalization())
    model.add(Dense(128, activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(Dense(1, activation='sigmoid')) 
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    evaluate_model(model, 'LSTM', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)

    #####################
    ### RANDOM FOREST ###
    #####################
    model = RandomForestClassifier(n_estimators=200, random_state=42)
    evaluate_model_RF(model, "RandomForest", train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)

    ##############
    ### ResNet ###
    ##############
    model = ResNet1D(train_data.shape[1], train_data.shape[2])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    evaluate_model(model, 'ResNet', train_data, train_labels, val_data, val_labels, test_data, test_labels, results_subdir)

## PREPROCESSING LOOP
Loop on defined parameters to generate all preprocessed data and compute first results.

Loop parameters can be changed in the first cell of the notebook.

In [ ]:
if os.path.exists(preprocessing_dir):
   shutil.rmtree(preprocessing_dir)
os.makedirs(preprocessing_dir)

if os.path.exists(results_dir):
   shutil.rmtree(results_dir)
os.makedirs(results_dir)

segments_dir_dict = {
    'seconds': sec_segments_dir,
    'beats': beats_segments_dir,
    'RR': RR_segments_dir,
    'pwave': pwave_segments_dir
}

#####################
for limited_time in limited_time_list:
    for loop_type in loop_list:
        for quality_type in quality_list:
            for analyse_type in analyse_list:
              print('Looping on :', analyse_type, loop_type, quality_type, limited_time)
              results_subdir = os.path.join(results_dir, f"{analyse_type}_{loop_type}_{quality_type}_{limited_time}")
              os.mkdir(results_subdir)
              subfolder = f"{analyse_type}_{loop_type}_{quality_type}"
              segments_dir = segments_dir_dict[analyse_type]
              preprocessing_subdir = os.path.join(segments_dir, f"{analyse_type}_{loop_type}_{quality_type}_{limited_time}")
              os.makedirs(preprocessing_subdir, exist_ok=True)
              train_dir = os.path.join(preprocessing_subdir, "train")
              val_dir = os.path.join(preprocessing_subdir, "val")
              test_dir = os.path.join(preprocessing_subdir, "test")
              os.makedirs(train_dir, exist_ok=True)
              os.makedirs(val_dir, exist_ok=True)
              os.makedirs(test_dir, exist_ok=True)

              
              #####################
              ### SECONDS model ###
              #####################
              if analyse_type == 'seconds':
                  afdb_total_segments, afdb_ignored_segments = process_ecg_data_seconds(afdb_dir, preprocessing_subdir, "afdb", is_afdb=True, seconds=seconds, limited_time=limited_time, loop_type=loop_type, quality_type=quality_type)
                  mitdb_total_segments, mitdb_ignored_segments = process_ecg_data_seconds(mitdb_dir, preprocessing_subdir, "mitdb", is_mitdb=True, seconds=seconds, limited_time=limited_time, loop_type=loop_type, quality_type=quality_type)
                  nsrdb_total_segments, nsrdb_ignored_segments = process_ecg_data_seconds(nsrdb_dir, preprocessing_subdir, "nsrdb", is_nsrdb=True, seconds=seconds, limited_time=limited_time, loop_type=loop_type, quality_type=quality_type)
                  if quality_type == 'excellent':
                    plot_segment_percentages(results_subdir,{'afdb': afdb_ignored_segments, 'mitdb': mitdb_ignored_segments, 'nsrdb': nsrdb_ignored_segments}, {'afdb': afdb_total_segments, 'mitdb': mitdb_total_segments, 'nsrdb': nsrdb_total_segments})
                  data_analysis(preprocessing_subdir, results_subdir)

              ####################
              ### BEATS model ####
              ####################
              if analyse_type == 'beats':
                  afdb_total_segments, afdb_ignored_segments = process_ecg_data_beats(afdb_dir, preprocessing_subdir, "afdb", is_afdb=True, n_beats=n_beats, limited_time=limited_time, loop_type=loop_type, quality_type=quality_type)
                  mitdb_total_segments, mitdb_ignored_segments = process_ecg_data_beats(mitdb_dir, preprocessing_subdir, "mitdb", is_mitdb=True, n_beats=n_beats, limited_time=limited_time, loop_type=loop_type, quality_type=quality_type)
                  nsrdb_total_segments, nsrdb_ignored_segments = process_ecg_data_beats(nsrdb_dir, preprocessing_subdir, "nsrdb", is_nsrdb=True, n_beats=n_beats, limited_time=limited_time, loop_type=loop_type, quality_type=quality_type)
                  if quality_type == 'excellent':
                    plot_segment_percentages(results_subdir,{'afdb': afdb_ignored_segments, 'mitdb': mitdb_ignored_segments, 'nsrdb': nsrdb_ignored_segments}, {'afdb': afdb_total_segments, 'mitdb': mitdb_total_segments, 'nsrdb': nsrdb_total_segments})
                  data_analysis(preprocessing_subdir, results_subdir)

              ################
              ### RR model ###
              ################
              if analyse_type == 'RR' and loop_type == 'manual' and quality_type == 'excellent':
                  process_ecg_data_RR(afdb_dir, preprocessing_subdir, "afdb", is_afdb=True, RRs=RRs, limited_time=limited_time)
                  process_ecg_data_RR(mitdb_dir, preprocessing_subdir, "mitdb", is_mitdb=True, RRs=RRs, limited_time=limited_time)
                  process_ecg_data_RR(nsrdb_dir, preprocessing_subdir, "nsrdb", is_nsrdb=True, RRs=RRs, limited_time=limited_time)
                  data_analysis(preprocessing_subdir, results_subdir)               

              #############
              ### PWAVE ###
              #############
              if analyse_type == 'pwave' and loop_type == 'manual' and quality_type == 'excellent':
                process_ecg_data_pwave(afdb_dir, preprocessing_subdir, "afdb", is_afdb=True, RRs=RRs, limited_time=limited_time)
                process_ecg_data_pwave(mitdb_dir, preprocessing_subdir, "mitdb", is_mitdb=True, RRs=RRs, limited_time=limited_time)
                process_ecg_data_pwave(nsrdb_dir, preprocessing_subdir, "nsrdb", is_nsrdb=True, RRs=RRs, limited_time=limited_time)
                data_analysis(preprocessing_subdir, results_subdir)
                

# MODELS LOOP

First get the data from the preprocessing folder.

Then run the models on the data, collect the results and plot the results.

In [ ]:
# Loop on all the preprocessing folders and run the models on them
# Loop parameters are defined in the first cell of the notebook
for root, dirs, files in os.walk(preprocessing_dir):
    for folder in dirs:
        if len(folder.split('_')) == 4:
            analyse_type, loop_type, quality_type, limited_time = folder.split('_')
            if analyse_type in analyse_list and loop_type in loop_list and quality_type in quality_list and int(limited_time) in limited_time_list:
                print('Looping on :', analyse_type, loop_type, quality_type, limited_time)
                results_subdir = os.path.join(results_dir, folder)
                os.makedirs(results_subdir, exist_ok=True)
                if analyse_type == 'seconds':
                    sec_run_models(seconds=seconds, results_subdir=results_subdir, actual_folder=folder)
                elif analyse_type == 'beats':
                    beats_run_models(results_subdir=results_subdir, actual_folder=folder)
                elif analyse_type == 'RR' and loop_type == 'manual' and quality_type == 'excellent':
                    RR_run_models(results_subdir=results_subdir, actual_folder=folder)
                elif analyse_type == 'pwave' and loop_type == 'manual' and quality_type == 'excellent':
                    pwave_run_models(results_subdir=results_subdir, actual_folder=folder)
                
collect_results(results_dir)
df = pd.read_csv(os.path.join(results_dir, 'complete_results.csv'))
plot_results(df, results_dir)